In [155]:
#import os
#import sys
#os.environ['PYSPARK_PYTHON'] = sys.executable
#os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

#import pyspark.pandas as pspd
import pandas as pd
coffe_df = pd.read_csv('/home/jovyan/data/CoffeeShopSales.csv', sep=';', header='infer')

<>:8: SyntaxWarning: invalid escape sequence '\D'
<>:8: SyntaxWarning: invalid escape sequence '\D'
C:\Users\fabio\AppData\Local\Temp\ipykernel_5508\2500401819.py:8: SyntaxWarning: invalid escape sequence '\D'
  coffe_df = pd.read_csv('C:\Data\CoffeeShopSales.csv', sep=';', header='infer')


In [156]:
coffe_df.head()

,transaction_id,transaction_date,transaction_time,transaction_qty,store_id,store_location,product_id,unit_price,product_category,product_type,product_detail
0,1,01/01/2023,07:06:11,2,5,Lower Manhattan,32,3,Coffee,Gourmet brewed coffee,Ethiopia Rg
1,2,01/01/2023,07:08:56,2,5,Lower Manhattan,57,"3,1",Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg
2,3,01/01/2023,07:14:04,2,5,Lower Manhattan,59,"4,5",Drinking Chocolate,Hot chocolate,Dark chocolate Lg
3,4,01/01/2023,07:20:24,1,5,Lower Manhattan,22,2,Coffee,Drip coffee,Our Old Time Diner Blend Sm
4,5,01/01/2023,07:22:41,2,5,Lower Manhattan,57,"3,1",Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg


# Transformações nos dados

In [157]:
coffe_df['unit_price'] = coffe_df['unit_price'].astype(str).str.replace(',', '.', regex=False).astype(float)

In [158]:
coffe_df['transaction_datetime'] = coffe_df['transaction_date'] + ' ' + coffe_df['transaction_time']

In [159]:
coffe_df['transaction_datetime'] = pd.to_datetime(coffe_df['transaction_datetime'], format='%d/%m/%Y %H:%M:%S')

In [160]:
coffe_df['transaction_date'] = pd.to_datetime(coffe_df['transaction_date'], format='%d/%m/%Y')

In [161]:
coffe_df = coffe_df.assign(
    total_value=coffe_df['unit_price'] * coffe_df['transaction_qty'],
)
coffe_df = coffe_df.round(decimals=2)

In [163]:
coffe_df.head()

,transaction_id,transaction_date,transaction_time,transaction_qty,store_id,store_location,product_id,unit_price,product_category,product_type,product_detail,transaction_datetime,total_value
0,1,2023-01-01,07:06:11,2,5,Lower Manhattan,32,3.0,Coffee,Gourmet brewed coffee,Ethiopia Rg,2023-01-01 07:06:11,6.0
1,2,2023-01-01,07:08:56,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg,2023-01-01 07:08:56,6.2
2,3,2023-01-01,07:14:04,2,5,Lower Manhattan,59,4.5,Drinking Chocolate,Hot chocolate,Dark chocolate Lg,2023-01-01 07:14:04,9.0
3,4,2023-01-01,07:20:24,1,5,Lower Manhattan,22,2.0,Coffee,Drip coffee,Our Old Time Diner Blend Sm,2023-01-01 07:20:24,2.0
4,5,2023-01-01,07:22:41,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg,2023-01-01 07:22:41,6.2


# Receita por Categoria de Produto
### Objetivo no Power BI: Criar gráficos de barras ou pizza para visualizar quais categorias geram maior receita

In [113]:
revenue_by_category = coffe_df.groupby('product_category')['total_value'].sum().reset_index()

In [114]:
revenue_by_category.sort_values(by='total_value', ascending=False)

,product_category,total_value
2,Coffee,269952.45
8,Tea,196405.95
0,Bakery,82315.64
4,Drinking Chocolate,72416.00
3,Coffee beans,40085.25
1,Branded,13607.00
6,Loose Tea,11213.60
5,Flavours,8408.80
7,Packaged Chocolate,4407.64


# Produtos Mais Vendidos
### Objetivo no Power BI: Criar rankings ou gráficos para destacar os produtos mais populares.

In [115]:
top_products = coffe_df.groupby('product_detail')['transaction_qty'].sum().reset_index()
top_products = top_products.sort_values(by='transaction_qty', ascending=False)

In [116]:
top_products.head()

,product_detail,transaction_qty
24,Earl Grey Rg,4708
20,Dark chocolate Lg,4668
54,Morning Sunrise Chai Rg,4643
47,Latte,4602
64,Peppermint Rg,4564


# Receita por Período do Dia
### Objetivo no Power BI: Visualizar padrões de consumo por horário (ex.: gráfico de linha ou barras).

In [117]:
def classify_time(row):
    hour = int(row.split(':')[0])
    if 6 <= hour < 12:
        return 'Manhã'
    elif 12 <= hour < 18:
        return 'Tarde'
    else:
        return 'Noite'

coffe_df['period_of_day'] = coffe_df['transaction_time'].apply(classify_time)

revenue_by_period = coffe_df.groupby('period_of_day')['total_value'].sum().reset_index()

In [118]:
revenue_by_period

,period_of_day,total_value
0,Manhã,388288.67
1,Noite,65668.52
2,Tarde,244855.14


# Ticket Médio por Loja
### Objetivo no Power BI: Comparar lojas com maior ticket médio.

In [119]:
avg_ticket_by_store = coffe_df.groupby('store_location')['total_value'].mean().reset_index()

In [120]:
avg_ticket_by_store

,store_location,total_value
0,Astoria,4.589891
1,Hell's Kitchen,4.661696
2,Lower Manhattan,4.814726


# Performance por Tipo de Produto
### Objetivo no Power BI: Verificar qual tipo de produto é mais rentável e popular.

In [121]:
performance_by_product_type = coffe_df.groupby('product_type').agg(
    total_revenue=('total_value', 'sum'),
    total_quantity=('transaction_qty', 'sum')
).reset_index()

In [122]:
performance_by_product_type.head()

,product_type,total_revenue,total_quantity
0,Barista Espresso,91406.20,24943
1,Biscotti,19793.53,5788
2,Black tea,2711.85,303
3,Brewed Black tea,47932.00,17462
4,Brewed Chai tea,77081.95,26250


# Receita Mensal
### Objetivo no Power BI: Criar gráficos de tendência ao longo do tempo (ex.: gráfico de linha).

In [123]:
coffe_df['month'] = pd.to_datetime(coffe_df['transaction_date']).dt.month

revenue_by_month = coffe_df.groupby('month')['total_value'].sum().reset_index()

In [124]:
revenue_by_month

,month,total_value
0,1,81677.74
1,2,76145.19
2,3,98834.68
3,4,118941.08
4,5,156727.76
5,6,166485.88


# Clientes Frequentes (Simulado)
### Objetivo no Power BI: Criar uma simulação de clientes com base nos IDs de transação, agrupando por frequência de compras fictícia e visualizar o comportamento de compras recorrentes.

In [125]:
# Simulando IDs de clientes a partir de `transaction_id`
coffe_df['customer_id'] = coffe_df['transaction_id'].apply(lambda x: f"Customer_{x % 100}")

frequent_customers = coffe_df.groupby('customer_id').agg(
    total_purchases=('transaction_id', 'count'),
    total_revenue=('total_value', 'sum')
).reset_index()
frequent_customers = frequent_customers.sort_values(by='total_purchases', ascending=False)


In [126]:
frequent_customers.head()

,customer_id,total_purchases,total_revenue
25,Customer_31,1494,7092.16
14,Customer_21,1494,6922.18
28,Customer_34,1494,6910.31
24,Customer_30,1494,7029.53
29,Customer_35,1494,6857.74


# Receita por Localização
### Objetivo no Power BI: Criar um mapa para visualizar a performance geográfica das lojas.

In [151]:
revenue_by_location = coffe_df.groupby('store_location')['total_value'].sum().reset_index()

In [152]:
revenue_by_location

,store_location,total_value
0,Astoria,232243.91
1,Hell's Kitchen,236511.17
2,Lower Manhattan,230057.25


# Análise de Outliers
### Objetivo no Power BI: Mostrar transações excepcionais ou possíveis erros.

In [129]:
mean_value = coffe_df['total_value'].mean()
std_value = coffe_df['total_value'].std()

coffe_df['is_outlier'] = coffe_df['total_value'].apply(
    lambda x: 'Outlier' if (x > mean_value + 3 * std_value or x < mean_value - 3 * std_value) else 'Normal'
)

In [130]:
coffe_df[['is_outlier', 'total_value']].head()

,is_outlier,total_value
0,Normal,6.0
1,Normal,6.2
2,Normal,9.0
3,Normal,2.0
4,Normal,6.2


# Taxa de Conversão por Produto
### Objetivo no Power BI: Identificar produtos com maior eficiência de vendas.

In [131]:
coffe_df['conversion_rate'] = coffe_df['transaction_qty'] / coffe_df['transaction_qty'].sum()

In [132]:
coffe_df[['conversion_rate', 'transaction_qty', 'product_detail']].head()

,conversion_rate,transaction_qty,product_detail
0,0.000009,2,Ethiopia Rg
1,0.000009,2,Spicy Eye Opener Chai Lg
2,0.000009,2,Dark chocolate Lg
3,0.000005,1,Our Old Time Diner Blend Sm
4,0.000009,2,Spicy Eye Opener Chai Lg


# Conversão para CSV
### Conversão dos dataframes em CSV

In [133]:
revenue_by_category.to_csv('C:/Data/Coffe_Sales_Revenue_by_Category.csv', index=True)
top_products.to_csv('C:/Data/Coffe_Sales_Top_Products.csv', index=True)
revenue_by_period.to_csv('C:/Data/Coffe_Sales_Revenue_by_Period.csv', index=True)
avg_ticket_by_store.to_csv('C:/Data/Coffe_Sales_AVG_Ticket_by_Store.csv', index=True)
performance_by_product_type.to_csv('C:/Coffe_Sales_Performance_by_Product_Type.csv', index=True)
revenue_by_month.to_csv('C:/Coffe_Sales_Revenue_by_Month.csv', index=True)
frequent_customers.to_csv('C:/Coffe_Sales_Frequent_Customers.csv', index=True)
revenue_by_location.to_csv('C:/Coffe_Sales_Revenue_by_Location.csv', index=True)
coffe_df.to_csv('C:/Coffe_Sales_and_Outlier_and_Conversion_Rate.csv', index=True)